In [18]:
# get match details and add these to postgres
import requests
import pandas as pd
import numpy as np
import psycopg2
import json
import time
import support_EventLogger as logger

In [14]:
process_name = 'ETL02_GetMatchDetails.ipynb' # for logger
#logger.log(process_name,'Process started, libraries loaded')

In [19]:
target_match = 7215991100

In [20]:
api_string = "https://api.opendota.com/api/matches/"+str(target_match)
#response = requests.get(api_string)
if response.status_code == 200:
    print("YAY!")
else:
    print("...")

YAY!


In [29]:
match = response.json()
match

{'match_id': 7215991100,
 'barracks_status_dire': 63,
 'barracks_status_radiant': 63,
 'chat': None,
 'cluster': 122,
 'cosmetics': None,
 'dire_score': 9,
 'dire_team_id': None,
 'draft_timings': None,
 'duration': 582,
 'engine': 1,
 'first_blood_time': 48,
 'game_mode': 22,
 'human_players': 10,
 'leagueid': 0,
 'lobby_type': 0,
 'match_seq_num': 6066913687,
 'negative_votes': 0,
 'objectives': None,
 'picks_bans': [{'is_pick': True, 'hero_id': 43, 'team': 1, 'order': 0},
  {'is_pick': True, 'hero_id': 28, 'team': 1, 'order': 1},
  {'is_pick': True, 'hero_id': 68, 'team': 0, 'order': 2},
  {'is_pick': True, 'hero_id': 17, 'team': 1, 'order': 3},
  {'is_pick': True, 'hero_id': 21, 'team': 0, 'order': 4},
  {'is_pick': True, 'hero_id': 27, 'team': 1, 'order': 5},
  {'is_pick': True, 'hero_id': 4, 'team': 0, 'order': 6},
  {'is_pick': True, 'hero_id': 59, 'team': 1, 'order': 7},
  {'is_pick': False, 'hero_id': 4, 'team': 0, 'order': 8},
  {'is_pick': False, 'hero_id': 75, 'team': 0, 'o

In [28]:
for key in match.keys():
    if str(type(match[key])) != "<class 'int'>":
        print(key,type(match[key]))

chat <class 'NoneType'>
cosmetics <class 'NoneType'>
dire_team_id <class 'NoneType'>
draft_timings <class 'NoneType'>
objectives <class 'NoneType'>
picks_bans <class 'list'>
radiant_gold_adv <class 'NoneType'>
radiant_team_id <class 'NoneType'>
radiant_win <class 'bool'>
radiant_xp_adv <class 'NoneType'>
skill <class 'NoneType'>
teamfights <class 'NoneType'>
version <class 'NoneType'>
players <class 'list'>
replay_url <class 'str'>


--
match_details table
From API
	(dire_score,radiant_score,first_blood_time,patch,region)
From code
	(num_player_account_ids) (0-10 int, for count of players with account_ids)
		Some player account_ids show up as None, might be privacy blocked?

Total scope:
	match_details(dire_score,radiant_score,first_blood_time,patch,region,num_player_account_ids)

gather player account IDs. If any single player lacks an ID, then make sure num_player_account_ids reflects that. Matches with num_player_account_ids = 10 have enough data for eventual logistic regression based on permutations of MMR that I can look up. Those without 10 account_ids will have gaps and thus aren't suited
for this version of logistic_regression. Still handy to have match/player data regardless, could use that for dashboarding

then load match details to postgres and load player_performance in a separate table only for those players with account_ids

--

player_performance table:
From API:
	(match_id,account_id, player_slot, deaths, denies, assists, gold_per_min, gold_spent, damage, damate_taken, hero_damage, hero_healing, hero_id, item_0, item_1, item_2, item_3, item_4, item_5, item_neutral, backpack_0, backpack_1, backpack_2, backpack_3, kill_streaks, kills, level, leaver_status, last_hits, net_worth, multi_kills, roshans_killed, rune_pickups, stuns, tower_damage, towers_killed, xp_per_min, isRadiant, kda, abandons)

Check those for lists. any lists should additionally be ignored. any None becomes 0.

Once all that is done, will additionally take the account_ids and do a nextbatch_accountid.csv situation with the 03getnewplayerdetails script. any new account_ids will get their MMR checked. this won't give MMR at time of match result though, so we can't see MMR over time and that will influence historic results... need to investigate this further.

In [ ]:
# 2023-06-25 starting with just one match from this list.
## Ultimately, there will be loops to accomplish each step.
#target_matches = []
#target_matches.append(public_match_results.iloc[0]['match_id']) # just grab the first match for now
#match_details = pd.DataFrame()

# the following code doesn't work. The length of Match dicts are inconsistent.
# will need to evaluate these inconsistencies deeper and then return here.
#for i,target_match in enumerate(target_matches):
#    # target matches are numpy.int64 by default
#    api_string = "https://api.opendota.com/api/matches/"+str(target_match)
#    response = requests.get(api_string)
##    if response.status_code == 200:
#        if i == 0:
#            match_details = pd.DataFrame.from_dict(response.json(), orient='columns') # arrays not of same length
#        else:
#            pass # to be replaced later with Concatenate
#    else:
#        print("Response status code: ", response.status_code)
#        break

In [ ]:
#response.json()
#response.json()['players'][0]
#response2.json()['players'][0]['account_id']
#need player details as well for MMR etc.
#response3 = requests.get("https://api.opendota.com/api/players/416336446")
#response3.json()
# There's the MMR and also the Rank Tier above. 
# #Will take some work to get 100 matches, then 100 match results,
# then 10 players per match result to get all MMRs and rank tiers.
# for spacing out API calls
#print('test')
#time.sleep(2)
#print('test2')